# 🚀 Пример работы с моделью ELV Rocket

Этот notebook демонстрирует использование линейной продольной модели ракеты-носителя (ELV - Expendable Launch Vehicle) в среде TensorAeroSpace.

## 📋 Что мы будем делать:
1. Импортируем необходимые библиотеки
2. Настроим временные параметры и опорный сигнал
3. Создадим и инициализируем среду ракеты
4. Выполним шаг симуляции с управляющим воздействием
5. Проанализируем результаты и внутреннее состояние модели

## 📚 Импорт библиотек

Загружаем все необходимые модули для работы с моделью ракеты-носителя:

In [2]:
# Основные библиотеки
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# Модули TensorAeroSpace для работы с ракетой
from tensoraerospace.envs import LinearLongitudinalELVRocket
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

## ⚙️ Настройка параметров симуляции

Определяем временные параметры и создаем ступенчатый опорный сигнал для управления ракетой:

In [3]:
# Параметры дискретизации
dt = 0.01  # Шаг дискретизации (секунды)

# Генерация временного периода для симуляции
tp = generate_time_period(tn=20, dt=dt)  # 20 секунд полета ракеты
tps = convert_tp_to_sec_tp(tp, dt=dt)    # Преобразование в секунды
number_time_steps = len(tp)             # Общее количество временных шагов

# Создание ступенчатого опорного сигнала
# Команда на изменение траектории подается на 10-й секунде полета
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

print(f'🚀 Параметры симуляции ракеты:')
print(f'   • Время полета: {tp[-1]:.1f} сек')
print(f'   • Шаг дискретизации: {dt} сек')
print(f'   • Количество шагов: {number_time_steps}')
print(f'   • Команда на маневр: {10} сек')

## 🛸 Создание и инициализация среды ракеты

Создаем среду ELV Rocket с заданными параметрами и выполняем инициализацию:

In [4]:
# Создание среды ракеты-носителя
env = gym.make(
    'LinearLongitudinalELVRocket-v0',
    number_time_steps=number_time_steps,     # Количество временных шагов
    initial_state=[[0],[0],[0]],             # Начальное состояние [скорость, угол, угловая_скорость]
    reference_signal=reference_signals       # Опорный сигнал для управления
)

# Инициализация среды и получение начального состояния
initial_state, info = env.reset()

print(f'✅ Среда ракеты успешно создана!')
print(f'🎯 Начальное состояние: {initial_state.flatten()}')
print(f'📊 Размерность пространства состояний: {env.observation_space.shape}')
print(f'🎮 Размерность пространства действий: {env.action_space.shape}')

# Возвращаем результат инициализации
(initial_state, info)

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(array([[0.],
        [0.],
        [0.]], dtype=float32),
 {})

## 🎮 Выполнение управляющего воздействия

Применяем команду управления двигателями ракеты и наблюдаем изменение состояния:

In [6]:
# Применение управляющего воздействия
control_command = np.array([[1]])  # Команда управления (тяга двигателей)

# Выполнение одного шага симуляции
state, reward, terminated, truncated, info = env.step(control_command)

print(f'🚀 Управляющая команда: {control_command.flatten()[0]}')
print(f'📈 Новое состояние ракеты: {state.flatten()}')
print(f'🏆 Полученная награда: {reward}')
print(f'🔚 Миссия завершена: {terminated}')
print(f'⏰ Симуляция прервана: {truncated}')
print(f'ℹ️ Дополнительная информация: {info}')

## 📊 Анализ результатов полета

Исследуем историю управляющих команд и опорные сигналы:

In [7]:
# Просмотр истории управляющих команд
print(f'📝 История команд управления двигателями:')
print(f'   • Первые команды: {env.unwrapped.model.store_input[0][:5]}')
print(f'   • Общее количество команд: {env.unwrapped.model.store_input.shape[1]}')

# Полная история команд
env.unwrapped.model.store_input

array([[1., 1., 0., ..., 0., 0., 0.]])

In [8]:
# Анализ опорного сигнала
print(f'🎯 Анализ опорного сигнала:')
print(f'   • Текущее значение (шаг 1): {env.unwrapped.reference_signal[0][1]}')
print(f'   • Форма сигнала: {env.unwrapped.reference_signal.shape}')
print(f'   • Тип сигнала: Ступенчатый (активация на 10-й секунде)')

# Значение опорного сигнала на втором временном шаге
env.unwrapped.reference_signal[0][1]

0.0

In [9]:
# Детальный анализ системы награды
print(f'🏆 Анализ системы награды:')
print(f'   • Значение награды: {reward}')
print(f'   • Тип данных: {type(reward)}')
print(f'   • Форма массива: {reward.shape if hasattr(reward, "shape") else "скаляр"}')
print(f'   • Интерпретация: Мера качества отслеживания опорной траектории')
print(f'   • Цель: Минимизация отклонения от заданного маршрута')

# Возвращаем значение награды
reward

array([0.1760814])

## 🎉 Заключение

В этом примере мы успешно продемонстрировали работу с моделью ракеты-носителя:

✅ **Создали среду ELV Rocket** с правильными параметрами
✅ **Настроили временные параметры** и опорный сигнал для управления
✅ **Выполнили симуляцию полета** с управляющими командами
✅ **Проанализировали результаты** и внутреннее состояние модели

### 🚀 Особенности модели ракеты:
- **3 состояния**: Скорость, угол наклона траектории, угловая скорость
- **Управление**: Команды тяги двигательной установки
- **Цель**: Точное следование заданной траектории полета

### 📊 Система награды:
- **Принцип**: Оценка качества отслеживания опорного сигнала
- **Оптимизация**: Минимизация отклонения от заданного маршрута
- **Применение**: Обучение автопилота ракеты

### 🎯 Практическое применение:
Эта модель может использоваться для:
- Разработки систем автоматического управления ракет
- Обучения алгоритмов с подкреплением для космических миссий
- Тестирования стратегий управления траекторией
- Симуляции различных сценариев полета

Модель готова для использования в сложных задачах управления космическими аппаратами! 🌌